### Model training
1. Split data into 70-30 for training/ validation set
2. Conduct feature engineering (e.g., adding new columns/ one hot encoding) on training and testing set **after** splitting of data to prevent data leakage
3. Using training set, do cross fold validation to select the best model (e.g., adaboost vs xgboost vs logistic regression) and best parameters
4. Using the best model obtained from step 2, train the best model using the full training dataset and validate using the validation dataset - tune the model using the validation dataset
5. Use the tuned model to predict on the unseen dataset that will be given and produce the dataset in the required output format

Note:
- When dealing with only drach sites, comment out the codes that talk about `Relative Position`
- When dealing with drach + non-drach sites, 2 approaches can be considered:
    - Consider `Relative Position` 
    - Do not consider relative positions - remember to drop `Position` variable

### Evaluation criteria:
Model will be evaluated on ROC AUC and PR AUC of validation dataset

**Make sure that ROC AUC is greater than 0.5 and PR AUC is above 0.04**

In [1]:
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

# Version 1 - Contains both DRACH and non-DRACH sites
df = pd.read_csv('./Data/full_df trial.csv')
df

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,Label
0,ENST00000000233,ENSG00000004059,243,AAGAC,4.223784,123.702703,0.008264,3.730,125.00,0.006970,0
1,ENST00000000233,ENSG00000004059,244,AGACC,7.382162,125.913513,0.009373,6.650,126.00,0.007970,0
2,ENST00000000233,ENSG00000004059,245,GACCA,4.386989,80.570270,0.007345,3.440,80.50,0.005980,0
3,ENST00000000233,ENSG00000004059,260,CAAAC,3.216424,109.681395,0.006609,2.880,110.00,0.005640,0
4,ENST00000000233,ENSG00000004059,261,AAACT,3.226535,107.889535,0.006813,3.000,108.00,0.005885,0
...,...,...,...,...,...,...,...,...,...,...,...
365509,ENST00000641834,ENSG00000167747,1537,TGACC,6.552982,123.263158,0.007419,5.790,124.00,0.006810,0
365510,ENST00000641834,ENSG00000167747,1538,GACCA,2.540877,82.289474,0.006472,2.330,82.00,0.006310,0
365511,ENST00000641834,ENSG00000167747,1692,TTGAC,4.090577,105.807692,0.008788,3.160,107.00,0.007090,0
365512,ENST00000641834,ENSG00000167747,1693,TGACA,8.702885,113.134615,0.006907,8.675,113.00,0.006705,0


In [2]:
# Obtain all unique gene names in the dataframe
gene_names = []
for i in df['Gene Name']:
    if i not in gene_names:
        gene_names.append(i)

# Split dataset into training and validation data (70/30 ratio) based on the Gene name
random.seed(4262)
training_genes = random.sample(gene_names, int(0.7 * len(gene_names)) ) # sample 70% of genes to be used in training
validation_genes = list(set(gene_names) - set(training_genes)) # remainder of genes that are not sampled will be used in validation
training_data = df[df['Gene Name'].isin(training_genes)]
validation_data = df[df['Gene Name'].isin(validation_genes)]

training_y = training_data['Label'].reset_index(drop=True)
training_X = training_data.drop(['Label'], axis=1).reset_index(drop=True)
validation_y = validation_data['Label'].reset_index(drop=True)
validation_X = validation_data.drop(['Label'], axis=1).reset_index(drop=True)

# One hot encoding for bases column of train and test set
training_X_dummies = pd.get_dummies(training_X['Bases'], drop_first=True)
training_X = pd.concat([training_X, training_X_dummies],axis=1)
validation_X_dummies = pd.get_dummies(validation_X['Bases'], drop_first=True)
validation_X = pd.concat([validation_X, validation_X_dummies],axis=1)

training_X

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,...,TAACC,TAACT,TAGAC,TGAAC,TGACA,TGACC,TGACT,TGGAC,TTAAC,TTGAC
0,ENST00000000233,ENSG00000004059,243,AAGAC,4.223784,123.702703,0.008264,3.730,125.00,0.006970,...,0,0,0,0,0,0,0,0,0,0
1,ENST00000000233,ENSG00000004059,244,AGACC,7.382162,125.913513,0.009373,6.650,126.00,0.007970,...,0,0,0,0,0,0,0,0,0,0
2,ENST00000000233,ENSG00000004059,245,GACCA,4.386989,80.570270,0.007345,3.440,80.50,0.005980,...,0,0,0,0,0,0,0,0,0,0
3,ENST00000000233,ENSG00000004059,260,CAAAC,3.216424,109.681395,0.006609,2.880,110.00,0.005640,...,0,0,0,0,0,0,0,0,0,0
4,ENST00000000233,ENSG00000004059,261,AAACT,3.226535,107.889535,0.006813,3.000,108.00,0.005885,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257293,ENST00000641784,ENSG00000284707,3243,GAACA,2.998125,97.346875,0.007648,2.675,97.60,0.006580,...,0,0,0,0,0,0,0,0,0,0
257294,ENST00000641784,ENSG00000284707,3244,AACAA,2.203750,88.439063,0.005190,2.130,88.70,0.004660,...,0,0,0,0,0,0,0,0,0,0
257295,ENST00000641784,ENSG00000284707,3265,CTAAC,1.874516,94.209677,0.005972,1.760,94.80,0.005065,...,0,0,0,0,0,0,0,0,0,0
257296,ENST00000641784,ENSG00000284707,3266,TAACT,2.194032,99.730645,0.006831,2.170,99.75,0.005785,...,0,1,0,0,0,0,0,0,0,0


In [3]:
validation_X

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,...,TAACC,TAACT,TAGAC,TGAAC,TGACA,TGACC,TGACT,TGGAC,TTAAC,TTGAC
0,ENST00000000412,ENSG00000003056,354,GAAAC,2.977180,108.360000,0.007340,2.495,109.00,0.005475,...,0,0,0,0,0,0,0,0,0,0
1,ENST00000000412,ENSG00000003056,355,AAACT,2.608600,106.584000,0.007782,2.635,108.00,0.006535,...,0,0,0,0,0,0,0,0,0,0
2,ENST00000000412,ENSG00000003056,356,AACTA,1.888520,94.174000,0.007045,1.835,94.15,0.006745,...,0,0,0,0,0,0,0,0,0,0
3,ENST00000000412,ENSG00000003056,366,GGGAC,3.961489,118.638298,0.008988,3.670,119.00,0.007970,...,0,0,0,0,0,0,0,0,0,0
4,ENST00000000412,ENSG00000003056,367,GGACC,6.045319,122.489362,0.007403,5.760,123.00,0.006930,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108211,ENST00000641834,ENSG00000167747,1537,TGACC,6.552982,123.263158,0.007419,5.790,124.00,0.006810,...,0,0,0,0,0,1,0,0,0,0
108212,ENST00000641834,ENSG00000167747,1538,GACCA,2.540877,82.289474,0.006472,2.330,82.00,0.006310,...,0,0,0,0,0,0,0,0,0,0
108213,ENST00000641834,ENSG00000167747,1692,TTGAC,4.090577,105.807692,0.008788,3.160,107.00,0.007090,...,0,0,0,0,0,0,0,0,0,1
108214,ENST00000641834,ENSG00000167747,1693,TGACA,8.702885,113.134615,0.006907,8.675,113.00,0.006705,...,0,0,0,0,1,0,0,0,0,0


### Feature engineering functions

In [4]:
# Obtain the counts of the individual bases and use them as features
def count_bases(bases):
    a,t,c,g=0,0,0,0
    for i in bases:
        if i == 'A':
            a+=1
        elif i == 'T':
            t+=1
        elif i == 'C':
            c+=1
        else:
            g+=1
    return a,t,c,g

In [5]:
# Relative positions
relative_positions = [1,2,3] * int(len(training_X)/3)
relative_positions_df = pd.DataFrame(relative_positions, columns=['Relative Position'])
training_X = pd.concat([training_X, relative_positions_df],axis=1)

training_X['Count_A'], training_X['Count_T'], training_X['Count_C'], training_X['Count_G'] = zip(*training_X['Bases'].apply(count_bases))
training_X = training_X.drop(['Bases'],axis=1)

training_X

,Transcript Name,Gene Name,Position,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,AAACA,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000000233,ENSG00000004059,243,4.223784,123.702703,0.008264,3.730,125.00,0.006970,0,...,0,0,0,0,0,1,3,0,1,1
1,ENST00000000233,ENSG00000004059,244,7.382162,125.913513,0.009373,6.650,126.00,0.007970,0,...,0,0,0,0,0,2,2,0,2,1
2,ENST00000000233,ENSG00000004059,245,4.386989,80.570270,0.007345,3.440,80.50,0.005980,0,...,0,0,0,0,0,3,2,0,2,1
3,ENST00000000233,ENSG00000004059,260,3.216424,109.681395,0.006609,2.880,110.00,0.005640,0,...,0,0,0,0,0,1,3,0,2,0
4,ENST00000000233,ENSG00000004059,261,3.226535,107.889535,0.006813,3.000,108.00,0.005885,0,...,0,0,0,0,0,2,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257293,ENST00000641784,ENSG00000284707,3243,2.998125,97.346875,0.007648,2.675,97.60,0.006580,0,...,0,0,0,0,0,2,3,0,1,1
257294,ENST00000641784,ENSG00000284707,3244,2.203750,88.439063,0.005190,2.130,88.70,0.004660,0,...,0,0,0,0,0,3,4,0,1,0
257295,ENST00000641784,ENSG00000284707,3265,1.874516,94.209677,0.005972,1.760,94.80,0.005065,0,...,0,0,0,0,0,1,2,1,2,0
257296,ENST00000641784,ENSG00000284707,3266,2.194032,99.730645,0.006831,2.170,99.75,0.005785,0,...,0,0,0,0,0,2,2,2,1,0


In [6]:
# Relative positions
relative_positions = [1,2,3] * int(len(validation_X)/3)
relative_positions_df = pd.DataFrame(relative_positions, columns=['Relative Position'])
validation_X = pd.concat([validation_X, relative_positions_df],axis=1)

validation_X['Count_A'], validation_X['Count_T'], validation_X['Count_C'], validation_X['Count_G'] = zip(*validation_X['Bases'].apply(count_bases)) 
validation_X = validation_X.drop(['Bases'],axis=1)

validation_X

,Transcript Name,Gene Name,Position,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,AAACA,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000000412,ENSG00000003056,354,2.977180,108.360000,0.007340,2.495,109.00,0.005475,0,...,0,0,0,0,0,1,3,0,1,1
1,ENST00000000412,ENSG00000003056,355,2.608600,106.584000,0.007782,2.635,108.00,0.006535,0,...,0,0,0,0,0,2,3,1,1,0
2,ENST00000000412,ENSG00000003056,356,1.888520,94.174000,0.007045,1.835,94.15,0.006745,0,...,0,0,0,0,0,3,3,1,1,0
3,ENST00000000412,ENSG00000003056,366,3.961489,118.638298,0.008988,3.670,119.00,0.007970,0,...,0,0,0,0,0,1,1,0,1,3
4,ENST00000000412,ENSG00000003056,367,6.045319,122.489362,0.007403,5.760,123.00,0.006930,0,...,0,0,0,0,0,2,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108211,ENST00000641834,ENSG00000167747,1537,6.552982,123.263158,0.007419,5.790,124.00,0.006810,0,...,1,0,0,0,0,2,1,1,2,1
108212,ENST00000641834,ENSG00000167747,1538,2.540877,82.289474,0.006472,2.330,82.00,0.006310,0,...,0,0,0,0,0,3,2,0,2,1
108213,ENST00000641834,ENSG00000167747,1692,4.090577,105.807692,0.008788,3.160,107.00,0.007090,0,...,0,0,0,0,1,1,1,2,1,1
108214,ENST00000641834,ENSG00000167747,1693,8.702885,113.134615,0.006907,8.675,113.00,0.006705,0,...,0,0,0,0,0,2,2,1,1,1


### Feature selection using Pearson Correlation

In [7]:
df_cor = training_X.drop(['Transcript Name', 'Gene Name'], axis=1)
cor_matrix = df_cor.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)] # Drop features if correlation is > 0.75
df_cor = df_cor.drop(df_cor[to_drop], axis=1)
df_cor

C:\Users\jingh\AppData\Local\Temp/ipykernel_17344/1420394356.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


,Position,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,AACAG,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,243,4.223784,123.702703,0.008264,0,0,0,0,0,0,...,0,0,0,0,0,1,3,0,1,1
1,244,7.382162,125.913513,0.009373,0,0,0,0,0,0,...,0,0,0,0,0,2,2,0,2,1
2,245,4.386989,80.570270,0.007345,0,0,0,0,0,0,...,0,0,0,0,0,3,2,0,2,1
3,260,3.216424,109.681395,0.006609,0,0,0,0,0,0,...,0,0,0,0,0,1,3,0,2,0
4,261,3.226535,107.889535,0.006813,0,0,1,0,0,0,...,0,0,0,0,0,2,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257293,3243,2.998125,97.346875,0.007648,0,0,0,0,0,0,...,0,0,0,0,0,2,3,0,1,1
257294,3244,2.203750,88.439063,0.005190,0,0,0,1,0,0,...,0,0,0,0,0,3,4,0,1,0
257295,3265,1.874516,94.209677,0.005972,0,0,0,0,0,0,...,0,0,0,0,0,1,2,1,2,0
257296,3266,2.194032,99.730645,0.006831,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,1,0


In [8]:
print(to_drop)

['Median SD', 'Median_Mean', 'Median Dwelling Time']


In [9]:
# Drop Position variable if not dealing with relative positions 
training_X = training_X.drop(training_X[to_drop], axis=1).drop(['Position'],axis=1)
validation_X = validation_X.drop(validation_X[to_drop], axis=1).drop(['Position'],axis=1)
training_X

,Transcript Name,Gene Name,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000000233,ENSG00000004059,4.223784,123.702703,0.008264,0,0,0,0,0,...,0,0,0,0,0,1,3,0,1,1
1,ENST00000000233,ENSG00000004059,7.382162,125.913513,0.009373,0,0,0,0,0,...,0,0,0,0,0,2,2,0,2,1
2,ENST00000000233,ENSG00000004059,4.386989,80.570270,0.007345,0,0,0,0,0,...,0,0,0,0,0,3,2,0,2,1
3,ENST00000000233,ENSG00000004059,3.216424,109.681395,0.006609,0,0,0,0,0,...,0,0,0,0,0,1,3,0,2,0
4,ENST00000000233,ENSG00000004059,3.226535,107.889535,0.006813,0,0,1,0,0,...,0,0,0,0,0,2,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257293,ENST00000641784,ENSG00000284707,2.998125,97.346875,0.007648,0,0,0,0,0,...,0,0,0,0,0,2,3,0,1,1
257294,ENST00000641784,ENSG00000284707,2.203750,88.439063,0.005190,0,0,0,1,0,...,0,0,0,0,0,3,4,0,1,0
257295,ENST00000641784,ENSG00000284707,1.874516,94.209677,0.005972,0,0,0,0,0,...,0,0,0,0,0,1,2,1,2,0
257296,ENST00000641784,ENSG00000284707,2.194032,99.730645,0.006831,0,0,0,0,0,...,0,0,0,0,0,2,2,2,1,0


In [10]:
training_X.columns

Index(['Transcript Name', 'Gene Name', 'Mean SD', 'Mean_Mean',
       'Mean Dwelling Time', 'AAACA', 'AAACC', 'AAACT', 'AACAA', 'AACAC',
       'AACAG', 'AACAT', 'AACCA', 'AACCC', 'AACCG', 'AACCT', 'AACTA', 'AACTC',
       'AACTG', 'AACTT', 'AAGAC', 'AGAAC', 'AGACA', 'AGACC', 'AGACT', 'AGGAC',
       'ATAAC', 'ATGAC', 'CAAAC', 'CAGAC', 'CGAAC', 'CGGAC', 'CTAAC', 'CTGAC',
       'GAAAC', 'GAACA', 'GAACC', 'GAACT', 'GACAA', 'GACAC', 'GACAG', 'GACAT',
       'GACCA', 'GACCC', 'GACCG', 'GACCT', 'GACTA', 'GACTC', 'GACTG', 'GACTT',
       'GAGAC', 'GGAAC', 'GGACA', 'GGACC', 'GGACT', 'GGGAC', 'GTAAC', 'GTGAC',
       'TAAAC', 'TAACA', 'TAACC', 'TAACT', 'TAGAC', 'TGAAC', 'TGACA', 'TGACC',
       'TGACT', 'TGGAC', 'TTAAC', 'TTGAC', 'Relative Position', 'Count_A',
       'Count_T', 'Count_C', 'Count_G'],
      dtype='object')

### K-fold validation 
Will be performing k cross-fold validation where k=10 using training dataset

In [11]:
# Break genes of training dataset up into n chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
# Obtain all unique genes in training dataset
unique_genes_training = []
for i in training_X['Gene Name']:
    if i not in unique_genes_training:
        unique_genes_training.append(i)
unique_genes_training2 = unique_genes_training.copy()        
unique_genes_training

In [13]:
random.seed(4262)
random.shuffle(unique_genes_training)
cross_val_data = chunks(unique_genes_training, round(len(unique_genes_training)/10)) # Will be using k=10 folds
cross_val_data = list(cross_val_data)

# Check to see what the cross_val_data looks like
for i in cross_val_data:
    print(i)
    print(len(i))
    print('')

In [14]:
training_X

,Transcript Name,Gene Name,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000000233,ENSG00000004059,4.223784,123.702703,0.008264,0,0,0,0,0,...,0,0,0,0,0,1,3,0,1,1
1,ENST00000000233,ENSG00000004059,7.382162,125.913513,0.009373,0,0,0,0,0,...,0,0,0,0,0,2,2,0,2,1
2,ENST00000000233,ENSG00000004059,4.386989,80.570270,0.007345,0,0,0,0,0,...,0,0,0,0,0,3,2,0,2,1
3,ENST00000000233,ENSG00000004059,3.216424,109.681395,0.006609,0,0,0,0,0,...,0,0,0,0,0,1,3,0,2,0
4,ENST00000000233,ENSG00000004059,3.226535,107.889535,0.006813,0,0,1,0,0,...,0,0,0,0,0,2,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257293,ENST00000641784,ENSG00000284707,2.998125,97.346875,0.007648,0,0,0,0,0,...,0,0,0,0,0,2,3,0,1,1
257294,ENST00000641784,ENSG00000284707,2.203750,88.439063,0.005190,0,0,0,1,0,...,0,0,0,0,0,3,4,0,1,0
257295,ENST00000641784,ENSG00000284707,1.874516,94.209677,0.005972,0,0,0,0,0,...,0,0,0,0,0,1,2,1,2,0
257296,ENST00000641784,ENSG00000284707,2.194032,99.730645,0.006831,0,0,0,0,0,...,0,0,0,0,0,2,2,2,1,0


In [15]:
roc_auc_scores = []
pr_auc_scores = []
for i in range(len(cross_val_data)):
    print(cross_val_data[i]) # The set that is selected will be used for validation
    # Train test split on training set
    training_cross_val = training_data[~training_data['Gene Name'].isin(cross_val_data[i])]
    validation_cross_val = training_data[training_data['Gene Name'].isin(cross_val_data[i])]
#     print(validation_cross_val)
    
    training_cross_val_y = training_cross_val['Label'].reset_index(drop=True)
    training_cross_val_X = training_cross_val.drop(['Label'], axis=1).reset_index(drop=True)
    validation_cross_val_y = validation_cross_val['Label'].reset_index(drop=True)
    validation_cross_val_X = validation_cross_val.drop(['Label'], axis=1).reset_index(drop=True)
    
    # Feature engineering
    training_cross_val_X['Top Bases Indicator'] = training_cross_val_X['Bases'].apply(indicate_top_bases)
    training_cross_val_X['Bottom Bases Indicator'] = training_cross_val_X['Bases'].apply(indicate_bottom_bases)
    training_cross_val_X['Combined Bases Indicator'] = training_cross_val_X['Top Bases Indicator'] + training_cross_val_X['Bottom Bases Indicator']
    training_cross_val_X['Count_A'], training_cross_val_X['Count_T'], training_cross_val_X['Count_C'] = zip(*training_cross_val_X['Bases'].apply(count_bases)) 
    training_cross_val_dummies = pd.get_dummies(training_cross_val_X['Bases'], drop_first=True)
    training_cross_val_X = pd.concat([training_cross_val_X, training_cross_val_dummies],axis=1).drop(['Bases'],axis=1)

    validation_cross_val_X['Top Bases Indicator'] = validation_cross_val_X['Bases'].apply(indicate_top_bases)
    validation_cross_val_X['Bottom Bases Indicator'] = validation_cross_val_X['Bases'].apply(indicate_bottom_bases)
    validation_cross_val_X['Combined Bases Indicator'] = validation_cross_val_X['Top Bases Indicator'] + validation_cross_val_X['Bottom Bases Indicator']
    validation_cross_val_X['Count_A'], validation_cross_val_X['Count_T'], validation_cross_val_X['Count_C'] = zip(*validation_cross_val_X['Bases'].apply(count_bases)) 
    validation_cross_val_dummies = pd.get_dummies(validation_cross_val_X['Bases'], drop_first=True)
    validation_cross_val_X = pd.concat([validation_cross_val_X, validation_cross_val_dummies],axis=1).drop(['Bases'],axis=1)

    # Drop Position variable when not dealing with relative positions
    training_cross_val_X = training_cross_val_X.drop(['Transcript Name', 'Gene Name'], axis=1).drop(['Position'],axis=1)
    validation_cross_val_X = validation_cross_val_X.drop(['Transcript Name', 'Gene Name'], axis=1).drop(['Position'],axis=1)
    print(validation_cross_val_X)
    
    # First apply SMOTE to bring minority class distribution to 10% of majority class then use RandomUnderSampler to bring majority class down
    # to 50 percent more than minority class before fitting model
    over = SMOTE(sampling_strategy=0.1)
    training_cross_val_X, training_cross_val_y = over.fit_resample(training_cross_val_X, training_cross_val_y)
    rus = RandomUnderSampler(random_state=4262, sampling_strategy = 0.5)
    training_cross_val_X, training_cross_val_y = rus.fit_resample(training_cross_val_X, training_cross_val_y)
    
    # Fitting ML model
    random.seed(4262)
    model = GradientBoostingClassifier(random_state=4262, max_depth=5)
    model.fit(training_cross_val_X, training_cross_val_y)
    y_pred = model.predict_proba(validation_cross_val_X)
    roc_auc_scores.append(roc_auc_score(validation_cross_val_y, y_pred[:,1], average=None))

    # Data to plot precision - recall curve
    precision, recall, thresholds = precision_recall_curve(validation_cross_val_y, y_pred[:,1])
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
    pr_auc_scores.append(auc_precision_recall)
    
print('Mean roc auc score is', np.mean(roc_auc_scores))
print('Mean pr auc score is', np.mean(auc_precision_recall))

### Manually Combine SMOTE and Random Undersampling
https://machinelearningmastery.com/combine-oversampling-and-undersampling-for-imbalanced-classification/

*Note that smote and random undersampling will be applied to training dataset only*

In [16]:
training_X = training_X.drop(['Transcript Name', 'Gene Name'], axis=1)
validation_X = validation_X.drop(['Transcript Name', 'Gene Name'], axis=1)

In [17]:
validation_X

,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,AACAG,AACAT,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,2.977180,108.360000,0.007340,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3,0,1,1
1,2.608600,106.584000,0.007782,0,0,1,0,0,0,0,...,0,0,0,0,0,2,3,1,1,0
2,1.888520,94.174000,0.007045,0,0,0,0,0,0,0,...,0,0,0,0,0,3,3,1,1,0
3,3.961489,118.638298,0.008988,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,3
4,6.045319,122.489362,0.007403,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108211,6.552982,123.263158,0.007419,0,0,0,0,0,0,0,...,1,0,0,0,0,2,1,1,2,1
108212,2.540877,82.289474,0.006472,0,0,0,0,0,0,0,...,0,0,0,0,0,3,2,0,2,1
108213,4.090577,105.807692,0.008788,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,2,1,1
108214,8.702885,113.134615,0.006907,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,1,1,1


In [18]:
validation_y

0         0
1         0
2         0
3         0
4         0
         ..
108211    0
108212    0
108213    0
108214    0
108215    0
Name: Label, Length: 108216, dtype: int64

In [19]:
# # Use RandomUnderSampler to bring majority class down to 5 percent more than minority class before fitting model
# over = SMOTE(sampling_strategy=0.1)
# training_X, training_y = over.fit_resample(training_X, training_y)
rus = RandomUnderSampler(random_state=4262, sampling_strategy = 0.05)
training_X, training_y = rus.fit_resample(training_X, training_y)
training_X

### Model training for Non-DRACH + DRACH sites

In [20]:
random.seed(4262)
clf = GradientBoostingClassifier(random_state=4262, max_depth=5)  # Change model here - e.g., AdaBoostClassifier, RandomForestClassifier
clf.fit(training_X, training_y)
y_score = clf.predict_proba(validation_X)
y_pred = clf.predict(validation_X)

In [21]:
y_score = pd.DataFrame(y_score[:,1], columns = ['Predicted Score'])
y_pred = pd.DataFrame(y_pred, columns=['Predicted'])
validation_y = validation_y.to_frame()
y_score

,Predicted Score
0,0.001230
1,0.076254
2,0.002626
3,0.001206
4,0.146576
...,...
108211,0.065469
108212,0.001123
108213,0.001281
108214,0.012550


In [22]:
validation_y

,Label
0,0
1,0
2,0
3,0
4,0
...,...
108211,0
108212,0
108213,0
108214,0


In [23]:
# Make sure that only DRACH sites are present before checking scores
y_pred_and_validation_y = pd.concat([y_score, validation_y, y_pred],axis=1)
indicator = pd.DataFrame([0,1,0] * int(len(y_pred_and_validation_y) / 3), columns = ['Indicator'])
y_pred_and_validation_y = pd.concat([y_pred_and_validation_y, indicator], axis=1)
y_pred_and_validation_y = y_pred_and_validation_y[y_pred_and_validation_y['Indicator'] == 1]
y_pred_and_validation_y = y_pred_and_validation_y.drop(['Indicator'], axis=1)
y_pred_and_validation_y

,Predicted Score,Label,Predicted
1,0.076254,0,0
4,0.146576,0,0
7,0.110033,0,0
10,0.145462,0,0
13,0.269892,0,0
...,...,...,...
108202,0.810625,1,1
108205,0.041885,0,0
108208,0.842805,1,1
108211,0.065469,0,0


In [24]:
# Convert DataFrame to Series before checking ROC AUC and PR AUC Score
validation_y = y_pred_and_validation_y['Label'].squeeze()
y_score = y_pred_and_validation_y['Predicted Score'].squeeze()
y_pred = y_pred_and_validation_y['Predicted'].squeeze()

In [25]:
print('ROC_AUC score is', roc_auc_score(validation_y, y_score, average=None))
# ROC_AUC score is 0.8665415772607338 (One hot encoding)

ROC_AUC score is 0.8693247433072782


In [26]:
# Data to plot precision - recall curve
precision, recall, thresholds = precision_recall_curve(validation_y, y_score)
# Use AUC function to calculate the area under the curve of precision recall curve
auc_precision_recall = auc(recall, precision)
print('PR AUC score is', auc_precision_recall)

PR AUC score is 0.41051980420315254


In [27]:
from sklearn.metrics import average_precision_score
average_precision_score(validation_y, y_score)

0.41068999165932707

In [28]:
from sklearn.metrics import precision_score, recall_score
print("Precision Score is", precision_score(validation_y, y_pred))
print("Recall Score is", recall_score(validation_y, y_pred))

Precision Score is 0.4861111111111111
Recall Score is 0.42706600110926235


In [29]:
# No undersampling
# Precision Score is 0.5927189988623436
# Recall Score is 0.2889628397115918

# Undersampling 5 percent
# Precision Score is 0.4882280049566295
# Recall Score is 0.4370493621741542

In [30]:
import pickle
pickle.dump(clf, open('gradientboostingclassifier.sav', 'wb'))

### Feature Importance
To check if certain features like `Top Bases Indicator` and `Bottom Bases Indicator` are contributing a lot to the model

In [31]:
clf.feature_importances_

array([1.86016893e-01, 2.41438489e-01, 4.63781494e-02, 0.00000000e+00,
       2.19634607e-04, 4.09431934e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.78143519e-19,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.31148330e-05,
       2.26628749e-03, 1.80128446e-04, 4.12382144e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.29015590e-03, 8.24602070e-04, 7.26800672e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.24637424e-09, 0.00000000e+00,
       0.00000000e+00, 4.04422893e-06, 5.12202261e-02, 3.98905073e-02,
       2.33361665e-01, 6.14195303e-05, 0.00000000e+00, 0.00000000e+00,
      

In [32]:
print(validation_X.columns)

Index(['Mean SD', 'Mean_Mean', 'Mean Dwelling Time', 'AAACA', 'AAACC', 'AAACT',
       'AACAA', 'AACAC', 'AACAG', 'AACAT', 'AACCA', 'AACCC', 'AACCG', 'AACCT',
       'AACTA', 'AACTC', 'AACTG', 'AACTT', 'AAGAC', 'AGAAC', 'AGACA', 'AGACC',
       'AGACT', 'AGGAC', 'ATAAC', 'ATGAC', 'CAAAC', 'CAGAC', 'CGAAC', 'CGGAC',
       'CTAAC', 'CTGAC', 'GAAAC', 'GAACA', 'GAACC', 'GAACT', 'GACAA', 'GACAC',
       'GACAG', 'GACAT', 'GACCA', 'GACCC', 'GACCG', 'GACCT', 'GACTA', 'GACTC',
       'GACTG', 'GACTT', 'GAGAC', 'GGAAC', 'GGACA', 'GGACC', 'GGACT', 'GGGAC',
       'GTAAC', 'GTGAC', 'TAAAC', 'TAACA', 'TAACC', 'TAACT', 'TAGAC', 'TGAAC',
       'TGACA', 'TGACC', 'TGACT', 'TGGAC', 'TTAAC', 'TTGAC',
       'Relative Position', 'Count_A', 'Count_T', 'Count_C', 'Count_G'],
      dtype='object')


In [33]:
print(len(clf.feature_importances_))

73


In [34]:
# Calculate the ranks of scores for feature importance
def calculate_rank(vector):
    a={}
    rank=1
    for num in sorted(vector):
        if num not in a:
            a[num]=rank
            rank=rank+1
    return[a[i] for i in vector]
calculate_rank(clf.feature_importances_)

[29,
 31,
 26,
 1,
 11,
 20,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 5,
 18,
 10,
 25,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 17,
 16,
 28,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 4,
 27,
 23,
 30,
 9,
 1,
 1,
 1,
 7,
 6,
 13,
 1,
 1,
 15,
 12,
 22,
 1,
 1,
 1,
 24,
 19,
 14,
 8,
 21]